In [93]:
import numpy as np
import pandas as pd
import Bio
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Align import substitution_matrices
from Bio import Align
from minineedle import needle, smith, core
from align import NeedlemanWunsch, read_fasta
import os
import difflib


__Test with implemented version__

In [2]:
# create instance
gap_open=-10
gap_extend=-1
nw=NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open, gap_extend)

# get sequences
human_brd2=read_fasta("data/Homo_sapiens_BRD2.fa")[0]
mus_brd2=read_fasta("data/Mus_musculus_BRD2.fa")[0]
gallus_brd2=read_fasta("data/Gallus_gallus_BRD2.fa")[0]
bal_brd2=read_fasta("data/Balaeniceps_rex_BRD2.fa")[0]
tur_brd2=read_fasta("data/Tursiops_truncatus_BRD2.fa")[0]

# get scores
print(nw.align(human_brd2, mus_brd2)[0])
print(nw.align(human_brd2, gallus_brd2)[0])
print(nw.align(human_brd2, bal_brd2)[0])
print(nw.align(human_brd2, tur_brd2)[0])

# print(s1[0],s2[0],s3[0],s4[0])

3682.0
3173.0
2941.0
3916.0


In [3]:
s1=pairwise2.align.globalds(human_brd2, mus_brd2, nw.sub_dict, gap_open+gap_extend, gap_extend)
s2=pairwise2.align.globalds(human_brd2, gallus_brd2, nw.sub_dict, gap_open+gap_extend, gap_extend)
s3=pairwise2.align.globalds(human_brd2, bal_brd2, nw.sub_dict, gap_open+gap_extend, gap_extend)
s4=pairwise2.align.globalds(human_brd2, tur_brd2, nw.sub_dict, gap_open+gap_extend, gap_extend)

print(s1[0].score)
print(s2[0].score)
print(s3[0].score)
print(s4[0].score)

3682.0
3173.0
2941.0
3916.0


In [58]:
# read fasta sequences
hs_seq, hs_header = read_fasta("./data/Homo_sapiens_BRD2.fa")
gg_seq, gg_header = read_fasta("./data/Gallus_gallus_BRD2.fa")

# create instance
gap_open=-10
gap_extend=-1
nw=NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open, gap_extend)

# score
hs_gg=nw.align(hs_seq, gg_seq)

# run pairwise2
hs_gg_pw2=pairwise2.align.globalds(hs_seq, gg_seq, nw.sub_dict, gap_open+gap_extend, gap_extend)


In [17]:
# for small sequences
seq1="MAVHQLIRRP" # test_seq3.fa
seq2="MQLIRHP" # test_seq4.fa
print(nw.align(seq1, seq2)[0])

seq1="MYQR"
seq2="MQR"
print(nw.align(seq1, seq2)[0])

alignments=pairwise2.align.globalds(seq1, seq2, nw.sub_dict, gap_open+gap_extend, gap_extend)
alignments[0].seqA
alignments[0].seqB
alignments[0].score

nw._align_matrix[len(seq2)][len(seq1)]


17.0
4.0


4.0

In [44]:
aligner=Align.PairwiseAligner()
aligner.open_gap_score=gap_open+gap_extend
aligner.extend_gap_score=gap_extend
aligner.substitution_matrix=substitution_matrices.load("BLOSUM62") 
alignments=aligner.align(seq1, seq2)
alignments=list(alignments)


__Initialization__

In [382]:
# example from provided video
# seq1="ACACT" # y sequence
# seq2="AAT" # x sequence

# gap open (h) and extension (g) penalties 
# gap_open=-3
# gap_extend=-1


# extraneous case
# seq1="ACAATCT" # y sequence
# seq2="AAT" # x sequence

# from provided files
# seq1="MAVHQLIRRP" # test_seq3.fa
# seq2="MQLIRHP" # test_seq4.fa

seq2="MYQR"
seq1="MQR"

# gap open (h) and extension (g) penalties 
gap_open=-10
gap_extend=-1


# initialize align matrix as mat of zeroes
align_matrix=np.zeros((len(seq2)+1, len(seq1)+1))
gapA_matrix=np.zeros((len(seq2)+1, len(seq1)+1))
gapB_matrix=np.zeros((len(seq2)+1, len(seq1)+1))

In [383]:
# initialize values

# first row and col of align matrix are -inf

# first row of align matrix
for j in (range(len(seq1)+1)):
    # align_matrix[0][j]=-np.inf
    align_matrix[0][j]=gap_open+gap_extend*j

# first column of align matrix
for i in (range(len(seq2)+1)):
    # align_matrix[i][0]=-np.inf
    align_matrix[i][0]=gap_open+gap_extend*i

# M(0,0)=0
align_matrix[0][0]=0

# first row of gapA
for j in (range(len(seq1)+1)):
    # gapA_matrix[0][j]=-np.inf
    gapA_matrix[0][j]=gap_open+gap_extend*j

# first column of gapA
for i in (range(len(seq2)+1)):
    gapA_matrix[i][0]=gap_open+gap_extend*i

# first column of gapB
for i in (range(len(seq2)+1)):
    # gapB_matrix[i][0]=-np.inf
    gapB_matrix[i][0]=gap_open+gap_extend*i

# first row of gapB
for j in (range(len(seq1)+1)):
    gapB_matrix[0][j]=gap_open+gap_extend*j 


# ADDED AFTER CHANGING -INF TO GAP OPEN OR EXTEND
# gapA_matrix[0][0]=0
# gapB_matrix[0][0]=0

__Alignmente Functions__

 - Functions were defined based on provided video [here](https://www.youtube.com/watch?v=NqYY0PJbD3s).

In [386]:
# def score(b1, b2):
#     s=-1
#     if (b1==b2):
#         s=1
#     return s

# ALTERNATE SCORE FUNCTION

def score(b1, b2):
    s=nw.sub_dict[(b1,b2)]
    return s

def fill_M(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2):
    M=align_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    Ix=gapA_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    Iy=gapB_matrix[i-1][j-1]+score(seq2[i-1], seq1[j-1])
    return max(M, Ix, Iy)

def fill_Ix(align_matrix, gapA_matrix, i, j, gap_open, gap_extend):
    M=align_matrix[i-1][j]+gap_open+gap_extend
    Ix=gapA_matrix[i-1][j]+gap_extend
    return max(M, Ix)

def fill_Iy(align_matrix, gapB_matrix, i, j, gap_open, gap_extend):
    M=align_matrix[i][j-1]+gap_open+gap_extend
    Iy=gapB_matrix[i][j-1]+gap_extend
    return max(M, Iy)

def fill_alignment_matrix(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2, gap_open, gap_extend):
    Mij=fill_M(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2)
    Ixij=fill_Ix(align_matrix, gapA_matrix, i, j, gap_open, gap_extend)
    Iyij=fill_Iy(align_matrix, gapB_matrix, i, j, gap_open, gap_extend)
    return Mij, Ixij, Iyij

In [387]:
# loop through indices of matrices
for i in range(1, len(seq2)+1):
    for j in range(1, len(seq1)+1):
        # print(i,j)
        align_matrix[i][j], gapA_matrix[i][j], gapB_matrix[i][j]=fill_alignment_matrix(align_matrix, gapA_matrix, gapB_matrix, i, j, seq1, seq2, gap_open, gap_extend)

In [388]:

gapA_matrix
gapB_matrix
align_matrix

# fill_M(align_matrix, gapA_matrix, gapB_matrix, 1, 1, seq1, seq2)

array([[  0., -11., -12., -13.],
       [-11.,   5., -11., -13.],
       [-12., -12.,   4.,  -8.],
       [-13., -12.,  -1.,   5.],
       [-14., -14.,  -6.,   4.]])

__Backtracing Functions__

 - Begin at the bottom right of the matrix and find the largest value among M, Ix, and Iy. 
 - If M is the largest value, we go left and up; if Ix is the largest we go up; and if Iy is the largest we go left.
 - Stop at top left of the matrix - (0,0).

In [313]:
# for the values across all three matrices at (i,j), get the max, and return an update vector that describes where to move
def get_next_move(align_matrix, gapA_matrix, gapB_matrix, i, j):
    mat_values={"M":align_matrix[i][j], "Ix":gapA_matrix[i][j], "Iy":gapB_matrix[i][j]}
    max_key=max(mat_values, key=mat_values.get)
    # check if there are multiple max values
    max_key_list=[key for key,val in mat_values.items() if val==mat_values[max_key]]
    return max_key_list

def query_sub_dict(match_val , mismatch_val, s1, s2, mtype="m", subdict={}): # mtypes: m, mm, sub - match, mismatch, substition
    if (mtype=="m"):
        return match_val
    if (mtype=="mm"):
        return mismatch_val
    if (mtype=="sub"):
        return subdict[(s1,s2)]

# WE MAY HAVE TO CHANGE SCORE FUNCTION ABOVE TO ALSO USE THE SUBSTITION VALUES FOR MATCH AND MISMATCH

def get_alignment_score(seq1_align, seq2_align, match_val, mismatch_val, gap_open, gap_extend, mtype="c", subdict={}):
    prev_label_seq1=None
    prev_label_seq2=None
    alignment_score=0
    alignment_print=[]
    for s1, s2 in zip(seq1_align, seq2_align):
        # if (mtype=="c"):
        if (s1!="-" and s2!="-"):
            if (s1==s2):
                # alignment_score+=match_val
                alignment_score+=subdict[(s1,s2)]
                alignment_print.append("|")
            else:
                # alignment_score+=mismatch_val
                alignment_score+=subdict[(s1,s2)]
                alignment_print.append("·")

            # ADDED TO RESET GAP LABEL
            prev_label_seq1=None
            prev_label_seq2=None

        else:
            if (s1=="-"):
                if (prev_label_seq1!="gap"):
                    alignment_score+=gap_open+gap_extend
                    prev_label_seq1="gap"
                else:
                    alignment_score+=gap_extend
            if (s2=="-"):
                if (prev_label_seq2!="gap"):
                    alignment_score+=gap_open+gap_extend
                    prev_label_seq2="gap"
                else:
                    alignment_score+=gap_extend
            alignment_print.append(" ")
        # else:


    return alignment_score, alignment_print

In [291]:
def print_alignment(seq1_align, seq2_align, alignment_print):
    print(''.join(seq1_align))
    print(''.join(alignment_print))
    print(''.join(seq2_align))

In [292]:
# starting positions are the current positions (bottom right corner) - pos variable is changed while start_pos is our record of the starting position
start_pos=(len(seq2), len(seq1)) # -> curr_i=len(seq2), curr_j=len(seq1)
pos=start_pos
print("Starting position: ", start_pos)

# alignment lists for sequences (they should be matching in length so we can draw 1 to 1 alignments)
seq1_align=[]
seq2_align=[]
update_vectors={"M":(-1,-1), "Ix":(-1,0), "Iy":(0,-1)} # where M -> go up left, Ix -> go up, Iy -> left

# method="highroad" 
method="lowroad"

# backtrace using the highroad method
# while (sum(pos)!=0):
while (pos[0]>0 and pos[1]>0):
    print(pos)
    max_key_list=get_next_move(align_matrix, gapA_matrix, gapB_matrix, pos[0], pos[1])

    # in the case of ties - for the highroad method, we prefer M; for the lowroad method, we prefer gaps
    if len(max_key_list)>1:
        if (method=="highroad"):
            max_key=max_key_list[0]
        if (method=="lowroad"):
            max_key=max_key_list[1]
    else:
        max_key=max_key_list[0]

    update_vector=update_vectors[max_key]

    pos=tuple(map(sum,zip(pos,update_vector)))
    # print(seq2[pos[0]], seq1[pos[1]])
    if (max_key=="M"):
        seq2_align.append(seq2[pos[0]])
        seq1_align.append(seq1[pos[1]])
    elif (max_key=="Ix"):
        seq2_align.append(seq2[pos[0]])
        seq1_align.append("-")
    elif (max_key=="Iy"):
        seq1_align.append(seq1[pos[1]])
        seq2_align.append("-")
        
print(pos)

# reverse lists
seq1_align.reverse()
seq2_align.reverse()


Starting position:  (3, 4)
(3, 4)
(2, 3)
(1, 2)
(1, 1)
(0, 0)


In [296]:
from align import NeedlemanWunsch, read_fasta
nw=NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open, gap_extend)
nw.align(seq1, seq2)

(3, 4)
(2, 3)
(1, 2)
(1, 1)


(4.0, 'MYQR', 'M-QR')

In [297]:
match_val=1
mismatch_val=-1
alignment_score, alignment_print=get_alignment_score(seq1_align, seq2_align, match_val, mismatch_val, gap_open, gap_extend, mtype="c", subdict=nw.sub_dict)
print(alignment_score)

4.0


In [298]:
print_alignment(seq1_align, seq2_align, alignment_print)

MYQR
| ||
M-QR


In [299]:
alignments=pairwise2.align.globalms(seq1, seq2, 1, -1, gap_open+gap_extend, gap_extend)
print(format_alignment(*alignments[0]))

MYQR
| ||
M-QR
  Score=-8



In [283]:
# testing from implementation
# seq1="MYQR"
# seq2="MQR"
from align import NeedlemanWunsch, read_fasta
nw=NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open, gap_extend)
nw.align(seq1, seq2)

(3, 4)
(2, 3)
(1, 2)
(1, 1)


(4.0, 'MYQR', 'M-QR')

In [284]:
mn_alignment=needle.NeedlemanWunsch(seq1, seq2)
mn_alignment.align()


In [96]:
gapA_matrix

array([[-10., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-11., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-12., -10., -23., -24., -25., -26., -27., -28., -29., -30., -31.],
       [-13., -11., -11., -22., -23., -22., -25., -26., -27., -28., -29.],
       [-14., -12., -12., -12., -23., -23., -21., -26., -27., -28., -29.],
       [-15., -13., -13., -13., -13., -24., -22., -20., -27., -28., -29.],
       [-16., -14., -14., -14., -14., -14., -23., -21., -19., -26., -29.],
       [-17., -15., -15., -15., -15., -15., -15., -22., -20., -20., -27.]])

In [105]:
# nmmat=np.array(mn_alignment._nmatrix)
# nmmat.shape
mn_alignment._alseq1

['M', 'A', 'V', 'H', 'Q', 'L', 'I', 'R', 'R', 'P']

In [86]:
mn_alignment.change_matrix(core.ScoreMatrix(match=1, miss=-1, gap=-10))



In [87]:
mn_alignment.get_almatrix()

[[0, -10, -20, -30, -40, -50, -60, -70, -80, -90, -100],
 [-10, 1, -9, -19, -29, -39, -49, -59, -69, -79, -89],
 [-20, -9, 0, -10, -20, -28, -38, -48, -58, -68, -78],
 [-30, -19, -10, -1, -11, -21, -27, -37, -47, -57, -67],
 [-40, -29, -20, -11, -2, -12, -22, -26, -36, -46, -56],
 [-50, -39, -30, -21, -12, -3, -13, -23, -25, -35, -45],
 [-60, -49, -40, -31, -20, -13, -4, -14, -24, -26, -36],
 [-70, -59, -50, -41, -30, -21, -14, -5, -15, -25, -25]]

In [73]:
nw._align_matrix

array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf,   1., -12., -13., -14., -15., -16., -17., -18., -19., -20.],
       [-inf, -12.,   0., -11., -12., -11., -14., -15., -16., -17., -18.],
       [-inf, -13., -11.,  -1., -12., -13., -10., -15., -16., -17., -18.],
       [-inf, -14., -12., -12.,  -2., -13., -14.,  -9., -16., -17., -18.],
       [-inf, -15., -13., -13., -13.,  -3., -14., -15.,  -8., -15., -18.],
       [-inf, -16., -14., -14., -12., -14.,  -4., -15., -16.,  -9., -16.],
       [-inf, -17., -15., -15., -15., -13., -15.,  -5., -16., -17.,  -8.]])

In [137]:
from Bio.Align import substitution_matrices
aligner = Bio.Align.PairwiseAligner()
aligner.open_gap_score = gap_open+gap_extend
aligner.extend_gap_score = gap_extend
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
alignments = aligner.align(seq1, seq2)
alignments = list(alignments)
alignment=alignments[0]


In [151]:
alignment

'MQLIRHP'

In [136]:
alignments[0]

<Alignment object (2 rows x 10 columns) at 0x1fe28162890>

In [123]:
for a in pairwise2.align.globalds(seq1, seq2, nw.sub_dict, gap_open+gap_extend, gap_extend):
    print(format_alignment(*a))
# pairwise2.align.globalds?

MAVHQLIRRP
|   ||||.|
M---QLIRHP
  Score=17



In [50]:
alignments=pairwise2.align.globalds(seq1, seq2, nw.sub_dict, gap_open+gap_extend, gap_extend)

In [74]:
# Bio.pairwise2.print_matrix(substitution_matrix)

In [ ]:
alignments=pairwise2.align.globalms(seq1, seq2, 1, -1, gap_open+gap_extend, gap_extend)
print(format_alignment(*alignments[0]))

In [59]:
nw._align_matrix

array([[  0., -inf, -inf, -inf, -inf],
       [-inf,   1., -12., -13., -14.],
       [-inf, -12.,   0.,  -9., -12.],
       [-inf, -13., -11.,  -1.,  -8.]])